In [3]:
from cluster import cluster as cl
rc = cl.start_cluster('homard', 14)

Waiting for connection file: /cita/h/home-2/dtamayo/.ipython/profile_default/security/ipcontroller-client.json


In [4]:
rc.ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [5]:
dv = rc[:]

In [6]:
%%px --local
maxorbs = 1.e7

import numpy as np
import rebound
import random
import os
import time

a1min = 0.04
a1max = 0.06
#a1 = 0.05 # semimajor axis of innermost planet in AU

mp = 5*3.e-6 # 5 earth masses
gamma = (2./3.*mp)**(1./3.) # Hill radius with factor of a taken out
imax = 1.*np.pi/180. # maximum inclination (uniform distribution)
emax = 2.e-2 # maximum eccentricity (uniform distribution)
betamin = 5.
betamax = 9.
Noutputs = 1000

kicksize=1.e-15

In [9]:
def system(var):
    sim_id, newseed = var
    random.seed(sim_id)
    
    a1 = random.uniform(a1min, a1max)
    beta1 = random.uniform(betamin, betamax)
    a2 = a1 + beta1*gamma*a1
    beta2 = random.uniform(betamin, betamax)
    a3 = a2 + beta2*gamma*a2
    
    sim = rebound.Simulation()
    sim.integrator="whfast"
    sim.G = 4*np.pi**2
    sim.exit_min_distance = a1*gamma # use smaller hill radius as exit condition
    
    
    sim.add(m=1.)
    sim.add(m=mp, a=a1, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=mp, a=a2, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=mp, a=a3, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.move_to_com()
    ps = sim.particles
    
    random.seed(newseed)
    ps[1].x *= 1. + random.uniform(-1,1)*kicksize
    ps[2].x *= 1. + random.uniform(-1,1)*kicksize
    ps[3].x *= 1. + random.uniform(-1,1)*kicksize

    P0 = ps[1].P
    tmax = maxorbs * P0 # number of inner planet orbital periods to integrate
    sim.dt = P0*0.07
    
    features = [maxorbs, mp, gamma]
    for p in ps[1:]:
        features = features + [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f, p.x, p.y, p.z, p.vx, p.vy, p.vz]
    
    stable = [True] # assume it's stable and update if not
    E0 = sim.calculate_energy()
    t0 = time.time()
    
    times = np.logspace(0,np.log10(tmax),Noutputs)
    
    try:
        for t in times:
            sim.integrate(t)
    except rebound.Encounter as error:
        pass

    features = stable+[sim.t/P0]+features+[np.abs((sim.calculate_energy()-E0)/E0), time.time()-t0]
    fname = '/mnt/raid-cita/dtamayo/Stability/data/distribution/distribution'+str(os.getpid())+'.csv'
    with open(fname, 'a') as f:
        f.write(str(sim.t/P0))
        f.write('\n')

In [40]:
sim_id=4283
Nsystems = 1000
newseeds = list(range(Nsystems))
sim_ids = [sim_id]*Nsystems
var = zip(sim_ids, newseeds)
res = dv.map_async(systemfeatures, var)

In [39]:
def systemfeatures(var):
    sim_id, newseed = var
    random.seed(sim_id)
    
    a1 = random.uniform(a1min, a1max)
    beta1 = random.uniform(betamin, betamax)
    a2 = a1 + beta1*gamma*a1
    beta2 = random.uniform(betamin, betamax)
    a3 = a2 + beta2*gamma*a2
    
    sim = rebound.Simulation()
    sim.integrator="whfast"
    sim.G = 4*np.pi**2
    sim.exit_min_distance = a1*gamma # use smaller hill radius as exit condition
    
    
    sim.add(m=1.)
    sim.add(m=mp, a=a1, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=mp, a=a2, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.add(m=mp, a=a3, e=random.random()*emax, pomega=random.random()*2.*np.pi, inc=random.random()*imax, Omega=random.random()*2.*np.pi, f=random.random()*2.*np.pi)
    sim.move_to_com()
    ps = sim.particles
    
    random.seed(newseed)
    ps[1].x *= 1. + random.uniform(-1,1)*kicksize
    ps[2].x *= 1. + random.uniform(-1,1)*kicksize
    ps[3].x *= 1. + random.uniform(-1,1)*kicksize

    P0 = ps[1].P
    tmax = maxorbs * P0 # number of inner planet orbital periods to integrate
    sim.dt = P0*0.07
    
    sim.init_megno()
    
    features = [maxorbs, mp, gamma]
    for p in ps[1:sim.N_real]:
        features = features + [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f, p.x, p.y, p.z, p.vx, p.vy, p.vz]
        
    stable = [True] # assume it's stable and update if not
    E0 = sim.calculate_energy()
    t0 = time.time()
    
    times = np.linspace(0,1.e4*P0,2000)
    outputs = '/mnt/raid-cita/dtamayo/Stability/data/distribution/outputs'+str(sim_id)+'/output'+str(newseed)+'.csv'

    with open(outputs, 'w') as f:
        try:
            for t in times:
                sim.integrate(t)
                f.write(str(sim.t))
                try:
                    outs = [1./sim.calculate_lyapunov()]
                except: 
                    outs = [np.inf]
                for p in ps[1:sim.N_real]:
                    outs = outs + [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]

                for out in outs:
                    f.write(',{0}'.format(out))
                f.write('\n')
        except rebound.Encounter as error:
            stable = [False]
            f.write(str(sim.t)) # this block added after the 1e7 dataset was generated for future use
            try:
                outs = [1./sim.calculate_lyapunov()]
            except: 
                outs = [np.inf]
            for p in ps[1:sim.N_real]:
                outs = outs + [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]

            for out in outs:
                f.write(',{0}'.format(out))
            f.write('\n')

    try:
        sim.integrate(1.e9*P0)
    except rebound.Encounter as error:
        stable = [False]
        
    features = stable+[sim.t/P0]+features+[np.abs((sim.calculate_energy()-E0)/E0), time.time()-t0]
    fname = '/mnt/raid-cita/dtamayo/Stability/data/distribution/outputs'+str(sim_id)+'/distribution'+str(os.getpid())+'.csv'
    with open(fname, 'a') as f:
        f.write(str(newseed))
        for feature in features:
            f.write(',{0}'.format(feature))
        f.write('\n')
    

In [41]:
res.ready()

True

In [42]:
import pandas as pd
import numpy as np
import os

columns = ['Sim_ID', 'Stable', 'Norbits_instability', 'Norbits', 'Mplanet', 'RH/a']
for i in ['1', '2', '3']:
    columns += ['a'+i, 'P'+i, 'e'+i, 'pomega'+i, 'inc'+i, 'Omega'+i, 'f'+i, 'x'+i, 'y'+i, 'z'+i, 'vx'+i, 'vy'+i, 'vz'+i]
columns += ['Rel_Eerr', 'integ_time']

frames = []
path = '/mnt/raid-cita/dtamayo/Stability/data/distribution/outputs'+str(sim_id)+'/'
for f in os.listdir(path):
        if f.startswith('distribution'):
            frames.append(pd.read_csv(path+f, names=columns, index_col=0))

df = pd.concat(frames)
df = df.sort_index()
df.tail()

,Stable,Norbits_instability,Norbits,Mplanet,RH/a,a1,P1,e1,pomega1,inc1,...,Omega3,f3,x3,y3,z3,vx3,vy3,vz3,Rel_Eerr,integ_time
Sim_ID,,,,,,,,,,,,,,,,,,,,,
995,False,4.220480e+06,10000000.0,0.000015,0.021544,0.045652,0.009754,0.013522,1.325028,0.004092,...,-1.946087,-4.701002,-0.04958,0.035818,-0.000597,-15.283265,-20.29633,-0.068275,0.000133,188.618895
996,False,2.797826e+06,10000000.0,0.000015,0.021544,0.045652,0.009754,0.013522,1.325028,0.004092,...,-1.946087,-4.701002,-0.04958,0.035818,-0.000597,-15.283265,-20.29633,-0.068275,0.000038,126.096676
997,False,2.938650e+06,10000000.0,0.000015,0.021544,0.045652,0.009754,0.013522,1.325028,0.004092,...,-1.946087,-4.701002,-0.04958,0.035818,-0.000597,-15.283265,-20.29633,-0.068275,0.000031,131.513661
998,False,1.744611e+07,10000000.0,0.000015,0.021544,0.045652,0.009754,0.013522,1.325028,0.004092,...,-1.946087,-4.701002,-0.04958,0.035818,-0.000597,-15.283265,-20.29633,-0.068275,0.006207,777.151159
999,False,2.035919e+06,10000000.0,0.000015,0.021544,0.045652,0.009754,0.013522,1.325028,0.004092,...,-1.946087,-4.701002,-0.04958,0.035818,-0.000597,-15.283265,-20.29633,-0.068275,0.001526,91.547254


In [43]:
sim_id

4283

In [44]:
df.to_csv(path+'distribution'+str(sim_id)+'.csv', encoding='ascii')

In [ ]:
cl.shutdown_cluster('homard', rc)